# Implementation - Belief Propagation

In [1]:
import numpy  as np
%run ./2-ImplementationFactor.ipynb
%run ./3-ImplementationPGM.ipynb

## 1 BP


In [2]:
class belief_propagation():
    def __init__(self, pgm):
        if type(pgm) is not factor_graph:
            raise Exception('PGM is not a factor graph')
        if not pgm.is_tree():
            raise Exception('PGM is not a tree')
        
        self.__msg = {}
        self.__pgm = pgm
        
    def __compute_variable2factor_msg(self, v_name, f_name):
        incoming_messages = []
        for f_name_neighbor in pgm.get_graph().vs[pgm.get_graph().neighbors(v_name)]['name']:
            if f_name_neighbor != f_name:
                incoming_messages.append(get_factor2variable_msg(f_name_neighbor, v_name))
        
        # if the variable does not have its own distribution
        if not incoming_messages:
            f_neighbor = pgm.get_graph().vs[pgm.get_graph().neighbors(f_name)]['factor_']
            order = f_neighbor.get_shape()[np.where(f_neighbor.get_shape()==v_name)[0][0]]
            incoming_messages = [factor(v_name, np.array([1]*order))]
        
        # Since all messages have the same dimension (1, order of v_name) the expression after
        # ```return``` is equivalent to ```factor(v_name, np.prod(incoming_messages))```
        return joint_distribution(incoming_messages)
    
    def __compute_factor2variable_msg(self, f_name, v_name):
        incoming_messages = [pgm.get_graph().vs[pgm.get_graph().neighbors(f_name)]['factor_']]
        marginalization_variables = []
        for v_name_neighbor in pgm.get_graph().vs[pgm.get_graph().neighbors(f_name)]['name']:
            if v_name_neighbor != v_name:
                incoming_messages.append(get_variable2factor_msg(v_name_neighbor, f_name))
                marginalization_variables.append(v_name_neighbor)
        return factor_marginalization(joint_distribution(incoming_messages), marginalization_variables) 
        
    def get_variable2factor_msg(self, v_name, f_name):
        key = (v_name, f_name)
        if key not in self.__msg:
            self.__msg[key] = self.__compute_variable2factor_msg(v_name, f_name)
        return self.__msg[key]
    
    def get_factor2variable_msg(self, f_name, v_name):
        key = (f_name, v_name)
        if key not in self.__msg:
            self.__msg[key] = self.__compute_factor2variable_msg(f_name, v_name)
        return self.__msg[key]
    
    def belief(self, v_name):
        incoming_messages = []
        for f_name_neighbor in pgm.get_graph().vs[pgm.get_graph().neighbors(v_name)]['name']:
            incoming_messages.append(get_factor2variable_msg(f_name_neighbor, v_name))
        return joint_distribution(incoming_messages)

#### Example

---

## 2 Loopy BP